In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import matplotlib.pyplot as plt
import seaborn as sns
from training_models import Autoencoder, DenoisingAutoencoder, AutoencoderEnsemble
from monitor_models import Autocovariance, AugmentationCrossCovariances, RepresentationCrossCovariances

In [ ]:
EXP_NAME = "exp_test"
TRAIN_DATASETS = ["instance", "stead"]
TEST_DATASETS = ["instance", "stead"]
OUTPUT_FILE_NAME = "summary.csv"

In [ ]:
def create_method_name(row):
    training_model_name = row["Training model"]
    monitoring_model_name = row["Monitoring model"]

    if training_model_name == Autoencoder().name and monitoring_model_name == Autocovariance().name:
        return "Autocovariance"
    
    if training_model_name == DenoisingAutoencoder().name and monitoring_model_name == Autocovariance().name:
        return "Autocovariance(Denoising)"
    
    if training_model_name == Autoencoder().name and monitoring_model_name == AugmentationCrossCovariances().name:
        return "Augmentation Cross-covariances"
    
    if training_model_name == DenoisingAutoencoder().name and monitoring_model_name == AugmentationCrossCovariances().name:
        return "Augmentation Cross-covariances(Denoising)"
    
    if training_model_name == AutoencoderEnsemble().name and monitoring_model_name == RepresentationCrossCovariances().name:
        return "Representation Cross-covariances"
    
def plot_rocauc_vs_epochs(df_grand_summary, figsize=(8, 6), ymin = 0.9, ymax=1.0):
    df_grand_summary["Method"] = df_grand_summary.apply(create_method_name)

    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=figsize)
    for train_idx, training_set in enumerate(TRAIN_DATASETS):
        for test_idx, test_set in enumerate(TEST_DATASETS):
            _df = df_grand_summary[(df_grand_summary["Training dataset"] == training_set) & (df_grand_summary["Testing dataset"] == test_set)]

            legend = (train_idx == 1) and (test_idx == 1) 
            sns.lineplot(data=_df, x='Epoch', y='ROC-AUC', hue='Method', ax=axs[train_idx, test_idx], legend=legend)

            axs[train_idx, test_idx].set_ylim(ymin, ymax)
            axs[train_idx, test_idx].set_title(f"Training dataset: {training_set.upper()}, Testing dataset: {test_set.upper()}")

            axs[train_idx, test_idx].get_xaxis().set_visible(train_idx == 1)

            if test_idx == 1:
                axs[train_idx, test_idx].set_ylabel("")

    plt.tight_layout(pad=0)
    plt.savefig('../evals/cross_domain.pdf', dpi=600)